In [84]:
#Import necessary packages
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder,DeployTransactionBuilder,MessageTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
import requests
import json
import pickle
import csv
import os

from iconsdk.libs.in_memory_zip import gen_deploy_data_content

In [10]:
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
SCORE_ADDRESS = "cxda0f8585cc7b40dec53b91a6e12849eac08f266a"
DEPLOY_PARAMS = {}

In [3]:
#Start ICON service and wallet
icon_service = IconService(HTTPProvider("https://bicon.net.solidwallet.io", 3))
#icon_service = IconService(HTTPProvider("http://localhost:9000", 3))

# Returns block information by block height
block = icon_service.get_block(1000)
block


{'version': '0.3',
 'height': 1000,
 'signature': 'Oj2VRhXDnd/3IdrE0ClPKEgGLkYLZt/p/o7D9rlspLoSAEA1vYSZyaOoF5n1QV8hpZaObMuZBGaRm13mjYmcHgA=',
 'prev_block_hash': 'cdd0017615d2af92227ec54d3147b199ba1417e1e6ec99cf33e7005a2b709692',
 'merkle_tree_root_hash': '0000000000000000000000000000000000000000000000000000000000000000',
 'time_stamp': 1586159775591512,
 'confirmed_transaction_list': [],
 'block_hash': '95ea963d180936776a650b6b35640c20c1ca951e26a0b55cb11b1b555ecb559b',
 'peer_id': 'hxaad52424d4aec9dac7d9f6796da527f471269d2c',
 'next_leader': 'hxec79e9c1c882632688f8c8f9a07832bcabe8be8f'}

In [4]:
operator = KeyWallet.load('config/yeouido/keystores/operator', 'bgsa199@#')
print(operator.get_address())
print(icon_service.get_balance(operator.get_address()))

hx8b94a3792f336b71937709dae5487166c180c87a
15324798278000000000000


In [8]:
DEPLOY_PARAMS =  {
            "_tokenName": "TestToken",
            "_symbolName": "TK",
			"_initialSupply": 1000,
            "_decimals": 18
		}
        
deploy_transaction = DeployTransactionBuilder()\
    .from_(operator.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('ODIContractsTest'))\
    .params(DEPLOY_PARAMS)\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, operator, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

'0x3478e8b67aaaab1666b22060aec67e52cd12596f5be606ca72016ec2f6f6f6bf'

In [11]:
tx_result = icon_service.get_transaction_result(tx_hash)
tx_result

{'txHash': '0x3478e8b67aaaab1666b22060aec67e52cd12596f5be606ca72016ec2f6f6f6bf',
 'blockHeight': 3598208,
 'blockHash': '0xbeec5d71d23cda3d551d75534eae8eac2fc541da42b2d4ac6608bda39ffc19b2',
 'txIndex': 1,
 'to': 'cx0000000000000000000000000000000000000000',
 'scoreAddress': 'cxda0f8585cc7b40dec53b91a6e12849eac08f266a',
 'stepUsed': 1201992120,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1201992120,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [73]:
UPDATE_PARAMS =  {
            "_tokenName": "TestToken1",
            "_symbolName": "TK1",
			"_initialSupply": 1000,
            "_decimals": 18
		}
        
deploy_transaction = DeployTransactionBuilder()\
    .from_(operator.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('ODIContractsTest'))\
    .params(UPDATE_PARAMS)\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, operator, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

'0xf93134b8e2142c2c56361c6cd1b4d44f9bdf0a1f92959f644c12dfdab63fa375'

In [75]:
tx_result = icon_service.get_transaction_result(tx_hash)
print(tx_result)

{'txHash': '0xf93134b8e2142c2c56361c6cd1b4d44f9bdf0a1f92959f644c12dfdab63fa375', 'blockHeight': 3599481, 'blockHash': '0x2fb1c72c2acf72277955f4c40806c07baf20e04f4baa5702d52b1f4fff5fa907', 'txIndex': 1, 'to': 'cxda0f8585cc7b40dec53b91a6e12849eac08f266a', 'scoreAddress': 'cxda0f8585cc7b40dec53b91a6e12849eac08f266a', 'stepUsed': 1802535560, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1802535560, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [76]:
external_methods = ["name", "symbol", "decimals", "totalSupply"]
for method in external_methods:
    call = CallBuilder().from_(operator.get_address())\
                    .to(SCORE_ADDRESS)\
                    .method(method)\
                    .build()
    print(icon_service.call(call))

TestToken1
TK1
0x12
0x3635c9adc5dea00000


In [86]:
burn_transaction = CallTransactionBuilder()\
    .from_(operator.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .method('burn')\
    .params({'_amount':1000})\
    .build()

estimate_step = icon_service.estimate_step(burn_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(burn_transaction, operator, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

Raised JSONRPCException while returning the custom response. Error message: {'code': -32001, 'message': "'bool' object is not callable"}


JSONRPCException: {'code': -32001, 'message': "'bool' object is not callable"} (Json rpc error)

In [80]:
tx_result = icon_service.get_transaction_result(tx_hash)
tx_result

{'txHash': '0xce005eb1f6c483ad195e8e95b7243d9e654efcbf608913d5516e8d94a1e4940e',
 'blockHeight': 3599511,
 'blockHash': '0x465827db0e476dfd57aee7712be912e50642b1f6e202250268f9de3077561a31',
 'txIndex': 1,
 'to': 'cxda0f8585cc7b40dec53b91a6e12849eac08f266a',
 'stepUsed': 135200,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 135200,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [ ]:
external_methods = ["totalSupply"]
for method in external_methods:
    call = CallBuilder().from_(operator.get_address())\
                    .to(SCORE_ADDRESS)\
                    .method(method)\
                    .build()
    print(icon_service.call(call))